In [1]:
from math import ceil, factorial, floor

import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
def erlang_c_prob_wait(N, A):
    """
    Erlang C: probability a caller has to wait.
    N: number of agents (float; we floor to int)
    A: offered load in Erlangs
    """
    N_int = int(np.floor(N))
    if N_int <= 0:
        return 1.0
    if A >= N_int:
        return 1.0  # overloaded system

    numer = (A**N_int / factorial(N_int)) * (N_int / (N_int - A))
    denom = sum(A**k / factorial(k) for k in range(N_int)) + numer
    return numer / denom


def erlang_c_asa(N, A, AHT):
    """
    ASA (Average Speed of Answer) in seconds.
    """
    N_int = int(np.floor(N))
    if N_int <= 0 or A >= N_int:
        return 1e9  # effectively infeasible (huge wait)

    P_wait = erlang_c_prob_wait(N_int, A)
    return (P_wait * AHT) / (N_int - A)


def occupancy(N, A):
    """
    Occupancy = Erlangs / # agents.
    """
    N_int = int(np.floor(N))
    if N_int <= 0:
        return 1.0
    return A / N_int

In [3]:
def find_min_staff_scipy(
    A,
    AHT,
    patience_sec=20,
    max_occ=0.85,
    N_min=1,
    N_max=200,
):
    """
    Find minimum integer N such that:
        ASA(N) <= patience_sec
        occupancy(N) <= max_occ
    Using SciPy (continuous) + local integer search.
    """

    # Objective: minimize headcount
    def obj(x):
        return x[0]

    # Constraints: ASA <= patience, Occ <= max_occ
    def asa_constraint(x):
        N = x[0]
        return patience_sec - erlang_c_asa(N, A, AHT)

    def occ_constraint(x):
        N = x[0]
        return max_occ - occupancy(N, A)

    cons = [
        {"type": "ineq", "fun": asa_constraint},
        {"type": "ineq", "fun": occ_constraint},
    ]
    bounds = [(N_min, N_max)]
    x0 = np.array([max(A + 1, N_min + 1)])  # start above load

    res = minimize(
        obj,
        x0,
        method="SLSQP",
        bounds=bounds,
        constraints=cons,
    )

    # If SciPy fails, we can fall back to a simple brute-force search
    if not res.success:
        best = None
        for N in range(N_min, N_max + 1):
            asa = erlang_c_asa(N, A, AHT)
            occ = occupancy(N, A)
            if asa <= patience_sec and occ <= max_occ:
                best = (N, asa, occ)
                break
        return best

    # Local integer search around relaxed optimum
    center = res.x[0]
    lo = max(N_min, int(floor(center)) - 5)
    hi = min(N_max, int(ceil(center)) + 5)

    best = None
    for N in range(lo, hi + 1):
        asa = erlang_c_asa(N, A, AHT)
        occ = occupancy(N, A)
        if asa <= patience_sec and occ <= max_occ:
            best = (N, asa, occ)
            break

    return best

In [4]:
def build_interval_demand(
    daily_path,
    intraday_path,
    interval_length_seconds=1800,
    normalize_intraday=True,
):
    """
    Expand daily + intraday data to interval-level traffic for:
    all Dates × all Product Groups × all 30-minute intervals.
    """
    daily = pd.read_csv(daily_path)
    intraday = pd.read_csv(intraday_path)

    # Add Day of Week to daily
    daily["Date"] = pd.to_datetime(daily["Date"])
    daily["Day of Week"] = daily["Date"].dt.day_name()

    # Optional: ensure intraday arrival fractions sum to 1 per PG+DOW
    if normalize_intraday:
        intraday["Intraday Arrival"] = intraday.groupby(
            ["Product Group", "Day of Week"]
        )["Intraday Arrival"].transform(
            lambda x: x / x.sum() if x.sum() > 0 else x
        )

    # Merge: Product Group + Day of Week
    merged = intraday.merge(
        daily,
        on=["Product Group", "Day of Week"],
        how="inner",
        suffixes=("_intraday", "_daily"),
    )

    # Interval-level volume + traffic
    merged["Interval_Length_sec"] = interval_length_seconds
    merged["Interval_Calls"] = (
        merged["Incoming Calls"] * merged["Intraday Arrival"]
    )
    merged["lambda"] = merged["Interval_Calls"] / interval_length_seconds
    merged["Traffic_Erlangs"] = (
        merged["lambda"] * merged["Talk Duration (AVG)"]
    )

    return merged

In [5]:
def optimize_staffing_for_date(
    merged,
    date,
    product_group=None,  # optional filter
    patience_sec=20,
    max_occ=0.85,
    N_min=1,
    N_max=200,
    zero_load_staff=0,  # policy for A <= 0
):
    """
    Optimize staffing for all intervals on a single date.
    Keeps ALL intervals (even zero-traffic ones).
    """
    # Normalize date
    date_ts = pd.to_datetime(date)

    day_data = merged[merged["Date"] == date_ts].copy()
    if product_group is not None:
        day_data = day_data[day_data["Product Group"] == product_group]

    if day_data.empty:
        return pd.DataFrame()

    results = []

    # Group by Product Group (even if only one)
    for pg, group in day_data.groupby("Product Group"):
        AHT = group["Talk Duration (AVG)"].iloc[0]

        for _, row in group.iterrows():
            A = row["Traffic_Erlangs"]

            # Defaults (for zero-load or failures)
            N_opt = None
            asa_opt = None
            occ_opt = None

            if A <= 0:
                # Zero offered load: use policy
                N_opt = zero_load_staff
                asa_opt = 0.0
                occ_opt = 0.0
            else:
                sol = find_min_staff_scipy(
                    A,
                    AHT,
                    patience_sec=patience_sec,
                    max_occ=max_occ,
                    N_min=N_min,
                    N_max=N_max,
                )
                if sol is not None:
                    N_opt, asa_opt, occ_opt = sol
                # else: remain None so you can see failures

            results.append(
                {
                    "Product Group": pg,
                    "Date": date_ts.date(),
                    "Day of Week": row["Day of Week"],
                    "Interval Start": row["Interval Start"],
                    "Interval_Length_min": row["Interval_Length_sec"] / 60,
                    "Interval_Calls": round(row["Interval_Calls"], 3),
                    "Traffic_Erlangs": round(A, 3),
                    "AHT_sec": round(AHT, 2),
                    "Optimal_Staff": None if N_opt is None else int(N_opt),
                    "ASA_sec": asa_opt,
                    "Occupancy": occ_opt,
                }
            )

    return pd.DataFrame(results)

In [7]:
def optimize_staffing_for_all_dates(
    merged,
    product_group=None,  # optional: limit to one PG
    patience_sec=20,
    max_occ=0.85,
    N_min=1,
    N_max=200,
    zero_load_staff=0,
):
    """
    Loop over all unique dates in `merged`,
    optionally restricted to a single Product Group.
    """
    all_dates = sorted(merged["Date"].dt.normalize().unique())
    all_results = []

    for d in all_dates:
        day_df = optimize_staffing_for_date(
            merged,
            date=d,
            product_group=product_group,
            patience_sec=patience_sec,
            max_occ=max_occ,
            N_min=N_min,
            N_max=N_max,
            zero_load_staff=zero_load_staff,
        )
        if not day_df.empty:
            all_results.append(day_df)

    if not all_results:
        return pd.DataFrame()

    return pd.concat(all_results, ignore_index=True)

In [8]:
daily_path = "call-center-data-v3-daily.csv"
intraday_path = "intraday-profiles.csv"

merged = build_interval_demand(
    daily_path=daily_path,
    intraday_path=intraday_path,
    interval_length_seconds=1800,
    normalize_intraday=True,
)

print("Merged shape:", merged.shape)  # should be Dates × 4 × 48

Merged shape: (17664, 12)


In [10]:
test_df = optimize_staffing_for_date(
    merged,
    date="2025-03-01",
    product_group="PRODUCT_ABC_DESKTOP_EN_CHAT",  # or None for all PGs
    patience_sec=20,
    max_occ=0.85,
    N_min=1,
    N_max=200,
    zero_load_staff=0,
)

print(test_df.shape)
test_df.head()

(48, 11)


,Product Group,Date,Day of Week,Interval Start,Interval_Length_min,Interval_Calls,Traffic_Erlangs,AHT_sec,Optimal_Staff,ASA_sec,Occupancy
0,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,00:00:00,30.0,0.0,0.0,184,0,0.0,0.0
1,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,00:30:00,30.0,0.0,0.0,184,0,0.0,0.0
2,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,01:00:00,30.0,0.0,0.0,184,0,0.0,0.0
3,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,01:30:00,30.0,0.0,0.0,184,0,0.0,0.0
4,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,02:00:00,30.0,0.0,0.0,184,0,0.0,0.0


In [11]:
test_df

,Product Group,Date,Day of Week,Interval Start,Interval_Length_min,Interval_Calls,Traffic_Erlangs,AHT_sec,Optimal_Staff,ASA_sec,Occupancy
0,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,00:00:00,30.0,0.000,0.000,184,0,0.000000,0.000000
1,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,00:30:00,30.0,0.000,0.000,184,0,0.000000,0.000000
2,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,01:00:00,30.0,0.000,0.000,184,0,0.000000,0.000000
3,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,01:30:00,30.0,0.000,0.000,184,0,0.000000,0.000000
4,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,02:00:00,30.0,0.000,0.000,184,0,0.000000,0.000000
5,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,02:30:00,30.0,0.000,0.000,184,0,0.000000,0.000000
6,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,03:00:00,30.0,0.000,0.000,184,0,0.000000,0.000000
7,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,03:30:00,30.0,0.000,0.000,184,0,0.000000,0.000000
8,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,04:00:00,30.0,0.000,0.000,184,0,0.000000,0.000000
9,PRODUCT_ABC_DESKTOP_EN_CHAT,2025-03-01,Saturday,04:30:00,30.0,0.000,0.000,184,0,0.000000,0.000000


| Column               | Type     | Description                                                                                                                                                                                                 |
|----------------------|----------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Product Group        | string   | Customer service segment or queue (e.g., desktop chat, mobile phone). Distinguishes traffic sources such as `PRODUCT_ABC_DESKTOP_EN_CHAT`, `PRODUCT_ABC_DESKTOP_EN_PHN`, etc.                               |
| Date                 | date     | Calendar date for the interval.                                                                                                                                                                             |
| Day of Week          | string   | Weekday name derived from `Date` (e.g., Monday). Used to match the correct intraday arrival profile.                                                                                                        |
| Interval Start       | string   | Start time of the 30-minute interval (`HH:MM:SS`). There are 48 intervals per date per product group.                                                                                                      |
| Interval_Length_min  | float    | Length of the interval in minutes. Typically `30.0`. Included to support verification in external Erlang-C calculators.                                                                                    |
| Interval_Calls       | float    | Expected number of calls arriving in the interval: daily incoming calls × intraday arrival % for that PG + DOW. Not necessarily an integer.                                                                 |
| Traffic_Erlangs      | float    | Offered load (in Erlangs): `(Interval_Calls / Interval_Length_sec) × AHT_sec`. Determines system load and required staffing.                                                                               |
| AHT_sec              | float    | Average handle time (seconds) for the product group and date (from the daily dataset).                                                                                                                     |
| Optimal_Staff        | int      | Minimum number of agents required such that ASA ≤ patience threshold and occupancy ≤ allowed max. Set to `0` for zero-traffic intervals (configurable policy).                                             |
| ASA_sec              | float    | Expected Average Speed of Answer (seconds), computed using Erlang-C and `Optimal_Staff`. Zero for zero-traffic intervals; may be `None` if optimization failed.                                           |
| Occupancy            | float    | Agent utilization given `Optimal_Staff`: `Traffic_Erlangs / Optimal_Staff`. Zero for zero-traffic intervals; may be `None` if optimization failed.                                                        |
